# Main working file

In [13]:
# imports

## system
import glob
import os
from pathlib import Path

## data
import numpy as np
import pandas as pd



# Preprocessing
Rewrapping: 


NOT IMPLEMENTED - NEED TO ASK JULIETTE / IVO

In [14]:
# Import the function
from util.video_transformation import rewrap_video

%load_ext autoreload
%autoreload 2 

path_source = r"data-in"
path_output =    r"data-in"

for team_folder in os.listdir(path_source):
    path_videos = os.path.join(path_source, team_folder, "VideoAudio")
    files = glob.glob(os.path.join(path_videos, "*.mp4"))

    for file in files:
        rewrap_video(file, path_output)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
data-in\05_06\VideoAudio\pp05_navigator_reconstructed_video_and_audio_discussion_phase_0.mp4
Video already has metadata
data-in\05_06\VideoAudio\pp05_navigator_reconstructed_video_and_audio_discussion_phase_1.mp4
Video already has metadata
data-in\05_06\VideoAudio\pp05_navigator_reconstructed_video_and_audio_instructional_video_0.mp4
Video already has metadata
data-in\05_06\VideoAudio\pp06_pilot_reconstructed_video_and_audio_discussion_phase_0.mp4
Video already has metadata
data-in\05_06\VideoAudio\pp06_pilot_reconstructed_video_and_audio_discussion_phase_1.mp4
Video already has metadata
data-in\05_06\VideoAudio\pp06_pilot_reconstructed_video_and_audio_instructional_video_0.mp4
Video already has metadata
data-in\05_06\VideoAudio\test.mp4
Video already has metadata
data-in\07_08\VideoAudio\pp07_navigator_reconstructed_video_and_audio_discussion_phase_0.mp4
Video already has metadata
data-in\07_08\Vid

# Extraction of Action Units (AUs)

In [15]:
from util.feature_extraction import extract_features
data_in = r'data-in'
data_out = r'data-out'

for pair in os.listdir(data_in):
    input_folder = os.path.join(data_in, pair)
    output_folder = os.path.join(data_out, pair, 'au')
    for participant in pair.split('_'):
        # TODO: handle missing data (NaN or recovered)
        aus = extract_features(input_folder, participant, output_folder)


pp05_navigator_instructional_video_0.csv has already been processed. Continuing...
pp05_navigator_discussion_phase_0.csv has already been processed. Continuing...
pp05_navigator_discussion_phase_1.csv has already been processed. Continuing...
pp06_pilot_instructional_video_0.csv has already been processed. Continuing...
pp06_pilot_discussion_phase_0.csv has already been processed. Continuing...
pp06_pilot_discussion_phase_1.csv has already been processed. Continuing...
pp07_navigator_instructional_video_0.csv has already been processed. Continuing...
pp07_navigator_discussion_phase_0.csv has already been processed. Continuing...
pp07_navigator_discussion_phase_1.csv has already been processed. Continuing...
pp08_pilot_instructional_video_0.csv has already been processed. Continuing...
pp08_pilot_discussion_phase_0.csv has already been processed. Continuing...
pp08_pilot_discussion_phase_1.csv has already been processed. Continuing...
pp09_navigator_instructional_video_0.csv has already

# Feature selection


In [22]:
location = r"data-out" 
for pair in os.listdir(location):
    filename = os.path.join(location, pair, "selection")
    os.makedirs(filename, exist_ok=True)

## Facial factors

In [ ]:
import util.feature_selection as fs
location = r"data-out" 
names = ['discussion_phase_0', 'discussion_phase_1', 'instructional_video_0']
for pair in os.listdir(location):
    # TODO: make it check which pair the file is from and make two dataframes of that. Then combine
    for file in os.listdir(os.path.join(location, pair, "au")):
        if ".csv" in file: 
            filename = os.path.join(location, pair, "au", file)
            p, _ = file.split("_",1)
            df = pd.read_csv(filename)
            for name in names:
                if name in file:
                    factors = fs.au_to_factors(df)
                    factors.to_csv(os.path.join(location, pair, "selection", f"{p}_{name}_factors.csv"))

## Correlated Component Analysis

In [34]:
import util.feature_selection as fs

# corrCA takes a df as input. This df should be all files for a pair. 
location = r"data-out" 
for pair in os.listdir(location):
    filename = os.path.join(location, pair)
    data = {}
    nav, pil = pair.split("_")
    nav_df = pd.DataFrame()
    pil_df = pd.DataFrame()
    for file in os.listdir(os.path.join(filename, "au")):
        if ".csv" in file: 
            df = pd.read_csv(os.path.join(filename, "au", file))
            if ".csv" in file and nav in file:
                nav_df = pd.concat([nav_df, df])
            if ".csv" in file and pil in file:
                pil_df = pd.concat([pil_df, df])
    fs.corrCA_weights(filename, nav_df, pil_df) #output = pair/corrca.csv

names = ['discussion_phase_0', 'discussion_phase_1', 'instructional_video_0']
for pair in os.listdir(location):
    # TODO: make it check which pair the file is from and make two dataframes of that. Then combine
    for file in os.listdir(os.path.join(location, pair, "au")):
        if ".csv" in file: 
            filename = os.path.join(location, pair, "au", file)
            p, _ = file.split("_", 1)
            df = pd.read_csv(filename)
            for name in names:
                if name in file:
                    w = pd.read_csv(os.path.join(location, pair, 'corrca_weights.csv'))
                    corrca = fs.apply_corrCA_weights(df, w)
                    corrca.to_csv(os.path.join(location, pair, "selection", f"{p}_{name}_corrca.csv"))
                    continue


files of pairs in data-out\05_06 do not have the same amount of datapoints - temporary fix has made them equal length
(17, 17)
files of pairs in data-out\07_08 do not have the same amount of datapoints - temporary fix has made them equal length
(17, 17)
(17, 17)
(9000,)
(9000,)
(18889,)
(9001,)
(9000,)
(18889,)
(8997,)
(8997,)
(18888,)
(9003,)
(9001,)
(18889,)
(9000,)
(9001,)
(18889,)
(9000,)
(9001,)
(18889,)


# Correlation measure

## Pearson
